# Stylized Facts of Cryptocurrency

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import glob
import re

plt.rcParams['figure.figsize'] = (15.0, 5.0)

In [ ]:
files = [f for f in glob.glob('../data/*price.csv')]
files.sort()
coin_names = list(map(lambda f: re.match(re.compile('.*/(.*)_price.csv'), f).group(1), files))
returns_map = dict();
data_sizes = pd.DataFrame(index=['size'], columns=coin_names)
for idx, file in enumerate(files):
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    prices = df['Close']
    returns = pd.Series(np.log(prices / prices.shift()))
    returns_map[coin_names[idx]] = returns;
    data_sizes[coin_names[idx]][0] = returns.size
data_sizes

## Basic Statistics
* Q1: What does A* mean, another version of central limit theorem?
* Q2: z-score here might be meaningless since it is used to assess the null hypothesis that the expected return is zero

In [ ]:
columns = ['10^4 mean', '10^2 std', 'max', 'min', 'skew', 'skew_abs', 'kurtosis', 'G%', 'A*%', 'z']
df = pd.DataFrame(index=coin_names, dtype='float', columns=columns)
N = 365;
for key, returns in returns_map.items():
    G = math.exp(N * returns.mean()) - 1
    Astar = (1 + G) * math.exp(N * 0.5 * math.pow(returns.std(), 2.0)) - 1
    z = returns.mean() * math.sqrt(returns.size) / returns.std() 
    
    df.loc[key] = [returns.mean() * 10000, returns.std() * 100, 
                   returns.max(), returns.min(), 
                   returns.skew(), abs(returns.skew()), returns.kurtosis(),
                   G * 100, Astar * 100, 
                   z]
df.round(2)

In [ ]:
ax = table.plot.scatter(x='kurtosis', y='skew_abs')
for i, text in enumerate(table.index):
    ax.annotate(text, (table['kurtosis'].iat[i], table['skew_abs'].iat[i]))
plt.show()

ax = table.plot.scatter(x='kurtosis', y='std')
for i, text in enumerate(table.index):
    ax.annotate(text, (table['kurtosis'].iat[i], table['std'].iat[i]))
plt.show()

ax = table.plot.scatter(x='kurtosis', y='mean')
for i, text in enumerate(table.index):
    ax.annotate(text, (table['kurtosis'].iat[i], table['mean'].iat[i]))
plt.show()

ax = table.plot.scatter(x='mean', y='std')
for i, text in enumerate(table.index):
    ax.annotate(text, (table['mean'].iat[i], table['std'].iat[i]))
plt.show()

ax = table.plot.scatter(x='min', y='max')
for i, text in enumerate(table.index):
    ax.annotate(text, (table['min'].iat[i], table['max'].iat[i]))
plt.show()

## Shape of Returns Distribution
Q: What's the threshold of skewness and kurtosis as a normal distribution.

In [ ]:
columns = ['std', 'skew', 'skew std', 'z-skew', 'kurtosis', 'kurtosis std', 'z-kurtosis']
df = pd.DataFrame(index=coin_names, dtype='float', columns=columns)
for key, returns in returns_map.items():
    skew_std = math.sqrt(6.0 / returns.size)
    kurto_std = math.sqrt(24.0 / returns.size)
    df.loc[key] = [returns.std(), 
                   returns.skew(), skew_std, abs(returns.skew()) / skew_std,
                   returns.kurtosis(), kurto_std, returns.kurtosis() / kurto_std]
df.round(2)

### Example shape graph

In [ ]:
returns = returns_map['bitconnect']

returns.plot('kde')
x = np.linspace(returns.min(), returns.max(), returns.size)
plt.plot(x, mlab.normpdf(x, returns.mean(), returns.std()))
returns.plot('hist', bins=40, secondary_y=True, alpha=0.5)
plt.show()

returns.plot()
plt.show()

## Calendar Effects

## Autocorrelation